# [GALAXY] - Image Reducer

The purpose of this notebook is to reduce the FLC files from Hubble by:

1. Aligning FLCs to the GAIA catalog
2. Drizzling Images together from a particular filter

## Imports

In [ ]:
# Python Imports
import os
from os import path
from glob import glob, iglob
import tarfile

# 3rd Party Imports
from matplotlib import pyplot as plt

# Astropy Colab Imports
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import QTable
from drizzlepac.tweakreg import TweakReg
from drizzlepac.astrodrizzle import AstroDrizzle
from astroscrappy import detect_cosmics as detectcrs

## Notebook Setup

In [ ]:
# Data Directory
DATA_DIR = 'mastDownload/HST'

# FLC Glob Patter
FLC_CR_GLOB_PAT = path.join(DATA_DIR, '**/*_deep_cr_fl?.fits')

## Load the Data

In [ ]:
# Get the File Names and Sort them by filter
fileNameDict = {}
for fn in iglob(FLC_CR_GLOB_PAT):
    
    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']
    
    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)

## Align Images to GAIA

### TweakReg Data Quality Flags

This cell defines the [DQ flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions) we want to ignore in the TweakReg process.

In [ ]:
# DQ Bits
DQ_BAD_DET  = 4
DQ_HOT_PIX  = 16
DQ_CR_PIX   = 4096+8192
DQ_GOOD_PIX = ~(DQ_BAD_DET + DQ_HOT_PIX + DQ_CR_PIX)

### Align First F475W Image to GAIA

In [ ]:
# Image Find Parameters
imagefindcfg = refimagefindcfg = dict(
    # peakmax=900,
    threshold=6.,
    conv_width=3.5,
    dqbits=DQ_GOOD_PIX
)

# Align the Images to the GAIA data
TweakReg(
    fileNameDict['F475W'][0],
    updatehdr=False,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refcat='../Data/GAIA/[GALAXY_SHORT]-GAIA-RefCatalog-icrs.txt',
    runfile='FirstToGAIA-Tweak.log',
    shiftfile=True,
    outshifts='FirstToGAIA-Shifts.log',
    outwcs='FirstToGAIA_wcs.fits',
    searchrad=0.25,
    minobj=5,
    tolerance=3,
    imagefindcfg=imagefindcfg
)

### Align the Rest of the Images to One Chosen Above

In [ ]:
# Image Find Parameters
imagefindcfg = refimagefindcfg = dict(
    # peakmax=900,
    threshold=10,
    conv_width=3.5,
    dqbits=DQ_GOOD_PIX
)

# Align the Images to the GAIA data
TweakReg(
    fileNameDict['F814W'] + fileNameDict['F475W'][1:],
    updatehdr=False,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refimage=fileNameDict['F475W'][0],
    runfile='RestToFirst-Tweak.log',
    shiftfile=True,
    outshifts='RestToFirst-Shifts.log',
    outwcs='RestToFirst_wcs.fits',
    searchrad=0.1,
    minobj=15,
    tolerance=1.00,
    imagefindcfg=imagefindcfg,
    refimagefindcfg=refimagefindcfg
)

In [ ]:
# Clean Up
!mkdir tweak/
!rm *.match
!rm *.coo
!rm *.list
!mv *.log tweak/
!mv *_wcs.fits tweak

## Drizzle Images for CR Correction

Although there will be additional notes added later, it is worth noting that according to
[STScI](https://hst-docs.stsci.edu/drizzpac/chapter-6-reprocessing-with-the-drizzlepac-package/6-3-running-astrodrizzle#id-6.3RunningAstroDrizzle-SelectingtheOptimalScaleandPixfrac):

1. For sub-pixel dithered data, select an output scale that's smaller than the native scale.
It will even help in the cosmic ray rejection step.
1. A smaller final_pixfrac gives higher resolution and lower correlated noise, but also reduces
sensitivity to low-surface brightness features (though it is possible to convolve a high resolution
image later to go after low surface brightness features).
1. Keep the standard deviation of the weight map over the main part of the image to above ~0.3 of
the mean to insure that one does not lose significant signal-to-noise in ignoring the weight map in
final photometry.

To summarize the last step, a `final_scale`/`final_pixfrac` combo should be chosen such that,
for the weight image,
\begin{equation}
    \mathrm{std} \gtrsim 0.3 \, \mathrm{mean}
\end{equation}

### AstroDrizzle ACS Data Quality Flags

[Link to ACS/WFC DQ Flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions)

In [ ]:
# DQ Bits
DQ_BAD_DET  = 4
DQ_STAB_HOT = 16
DQ_WARM_PIX = 64
DQ_BAD_COL  = 128
DQ_FULL_WELL= 256
DQ_BADREF   = 512
DQ_SINK_PIX = 1024
DQ_GOOD_PIX = DQ_STAB_HOT + DQ_WARM_PIX + DQ_BAD_COL + DQ_FULL_WELL + DQ_SINK_PIX
# DQ_GOOD_PIX = ~4096

### Drizzle F814W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F814W'],
    output='[GALAXY_SHORT]-F814W',
    runfile='F814W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=1,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_rot=0,
    final_scale=0.03
)

### Drizzle F475W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F475W'],
    output='[GALAXY_SHORT]-F475W',
    runfile='F475W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=1,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_refimage='[GALAXY_SHORT]-F814W_drc.fits'
)

In [ ]:
# Clean Up
!mkdir astro
!mkdir -p ProcessedImages/HST/DrizzledImages
!mv *Astro.log astro/
!mv *drc.fits ProcessedImages/HST/DrizzledImages/